# Chapter 7: Classes and Interfaces

# Item 48: Accept Functions Instead of Classes for Simple Interfaces
Sometimes a class is too much man. Python treats functions as first-class citizens, and sometimes passing a function is enough.

Python's `list` has a sort method that takes a function, behold

In [2]:
names.sort(key=len)# I
names

['Luke', 'Jimmy', 'Blake', 'Andrew']

We passed the `len` function to the interface. This told the list to sort from shortest to longest name!

In Java we would have to make a class to do this same thing:
```java
class SortByLength implements Comparator<String> 
{    
    public int compare(String a, String b) {
        return a.length() < b.length();
    }
}

// OR

names.sort((first, second) -> Integer.compare(first.length(), second.length())
```

### Hooks
this type of interaction in programming is called a "hook"

In [4]:
# I
def log_missing():
    print("key added!")
    return 0


from collections import defaultdict

some_dict = {"green": 12, "blue": 3}

defaulted_dict = defaultdict(log_missing, some_dict)

defaulted_dict["green"] += 3
defaulted_dict["red"] += 5
defaulted_dict["blue"] += 7
dict(defaulted_dict)

key added!


{'green': 15, 'blue': 10, 'red': 5}

### But why do this?
This basically gives you a clean way to trigger side-effects and keep them separate from your deterministic behavior.

In [7]:
# I

def count_missing():
    global missed  # remember nonlocal?
    missed += 1
    return 0


dict2 = defaultdict(count_missing, some_dict)
dict2["green"] += 3
dict2["orange"] += 2
dict2["purple"] += 1

print(missed)

2


Anyway, this is the power of first-class functions. Without this, you'd have to modify/extend `defaultdict` to get this behavior.

Lets do that example again, but without the gross `global` for state management.

In [9]:
    def __init__(s# Ielf):
        self.missed = 0

    def __call__(self):
        self.missed += 1
        return 0


my_cool_counter = Counterater()

dict2 = defaultdict(my_cool_counter,
                    some_dict)  # though I used __call__, you can also pass a class instance's method too.
dict2["green"] += 3
dict2["orange"] += 2
dict2["purple"] += 1

my_cool_counter.missed

2

### SURPRISE QUIZ: wtf is `__call__`?!
![spongebob.png](spongebob.png)

### Remember
Python functions are first-class, and thus allow composition, chaining, hooks, etc. Though a class to handle all the above may be a natural instinct, its not necessary in python.

# Item 49: Prefer Object-Oriented Polymorphism over Functions with `isinstance` Checks
TODO: sassy comment

In [12]:
class Integer:# I
    def __init__(self, value):
        self.value = value


class Add:
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Multiply:
    def __init__(self, left, right):
        self.left = left
        self.right = right


tree = Add(Integer(1), Integer(2))

In [14]:
# EVAL THAT TREE BABY# I

def evaluate(node):
    if isinstance(node, Integer):
        return node.value
    elif isinstance(node, Add):
        return evaluate(node.left) + evaluate(node.right)
    elif isinstance(node, Multiply):
        return evaluate(node.left) * evaluate(node.right)
    else:
        raise NotImplementedError

In [18]:
print(evaluate(Add(In# Iteger(11), Integer(22))))
print(evaluate(
    Add(Multiply(Integer(2), Integer(2)), Integer(22))))  # it even supports arbitrary nesting, since it's recursive!

3
33
26


Works great! But imagine we wanted to make our AST much bigger than just Add and Multiply? Gosh that `if..elif` structure would get pretty huge...

Let's make this more OOP-y

In [19]:
    def evaluate(self):
        raise NotImplementedError


class IntegerNode(Node):
    def __init__(self, value):
        self.value = value

    def evaluate(self):
        return self.value


class AddNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() + self.right.evaluate()


class MultiplyNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() * self.right.evaluate()

In [21]:
tree.evaluate()

3

This allows polymorphic extension later via abstracts etc. Though, it's easy for this to become its own maintenance nightmare. Care and consideration should be had!

# Item 50: Consider `functools.singledispatch` for Functional-Style Programming Instead of Polymorphism

In [22]:

class NodeAlt:
    def evaluate(self):
        raise NotImplementedError

    def pretty(self):
        raise NotImplementedError


class IntegerNodeAlt(NodeAlt):
    def __init__(self, value):
        self.value = value

    def evaluate(self):
        return self.value

    def pretty(self):
        return repr(self.value)


class AddNodeAlt(NodeAlt):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() + self.right.evaluate()

    def pretty(self):
        left_str = self.left.pretty()
        right_str = self.right.pretty()
        return f"({left_str} + {right_str})"


class MultiplyNodeAlt(NodeAlt):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def evaluate(self):
        return self.left.evaluate() * self.right.evaluate()

    def pretty(self):
        left_str = self.left.pretty()
        right_str = self.right.pretty()
        return f"({left_str} * {right_str})"

In [24]:
    AddNodeAlt(IntegerNodeAlt(3), IntegerNodeAlt(5)),
    AddNodeAlt(IntegerNodeAlt(4), IntegerNodeAlt(7)),
)
tree.pretty()

'((3 + 5) * (4 + 7))'

Do you see where this can go wrong? What If I need to add 30 different methods to `NodeAlt`? I have to update EVERY SINGLE INHERITOR! BOO! That sucks!!!

As a fun aside: Effective Java also talks about this exact problem, and urges programmers to use `abstract` sparringly.

In [31]:
from functools import singledispatch


@singledispatch
def pretty(value):
    raise NotImplementedError

Single-dispatch is a functional technique that lets you basically add behavior to a class without modifying it. The function above will be treated as the function of last resort, if nothing else can be found.

In [32]:
def _(value):  # note the name, _ basically means "the name doesn't matter"
    return repr(value)


@pretty.register(float)
def _(value):
    return f"{value:.2f}"

In [33]:
print(pretty(3))

3.14
3


In [35]:

@singledispatch
def evaluate(node):
    raise NotImplementedError


class Integer:
    def __init__(self, value):
        self.value = value


class BinaryOp:
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Add(BinaryOp):
    def __init__(self, left, right):
        super().__init__(left, right)


class Multiply(BinaryOp):
    def __init__(self, left, right):
        super().__init__(left, right)


@evaluate.register(Integer)
def _(node):
    return node.value


@evaluate.register(Add)
def _(node):
    return evaluate(node.left) + evaluate(node.right)


@evaluate.register(Multiply)
def _(node):
    return evaluate(node.left) * evaluate(node.right)

# HEY DONT FORGET PRETTY~
@pretty.register(Integer)
def _(node):
    return repr(node.value)

@pretty.register(Add)
def _(node):
    left_str = pretty(node.left)
    right_str = pretty(node.right)
    return f"({left_str} + {right_str})"

@pretty.register(Multiply)
def _(node):
    left_str = pretty(node.left)
    right_str = pretty(node.right)
    return f"({left_str} * {right_str})"

In [36]:
print(evaluate(tree))
print(pretty(tree))

21
((1 + 2) * (3 + 4))


An immediate benefit of this, is how the dispatch works with sub-classing.

In [37]:
    def __init__(self, value):
        if value < 0:
            raise ValueError("value must be positive")
        super().__init__(value)

pretty(PositiveInteger(3))

'3'

However, things get harder when we create a brand new class

In [38]:
    def __init__(self, value):
        self.value = value

pretty(Float(3.141592653589793))

NotImplementedError: 

This is a trade-off.
- Polymorphism: easy to add new classes, PITA to add new behavior across classes.
- Single Dispatch: easy to add new behavior across classes, PITA to add new classes.

From experience, the former is the *bigger* PITA. Single dispatch allows classes themselves to remain much simpler, and allows you as the programmer, to have independent systems with their own behaviors, use those classes without polluting them.

To solidify: look at Video Center and `abstract`; this is polymorphism, and it's awful. A single change causes cascading maintenance headaches everywhere. As a result, changes are seldom made, and the code-base has aged horribly.

But polymorphism still makes sense in smaller contexts, or fairly self-contained contexts, where the classes don't need to be shared across system boundaries.

# Item 51: Prefer `dataclasses` for Defining Lightweight Classes

In [51]:
from dataclasses import dataclass

@dataclass
class MyRGB:
    red: int
    green: int
    blue: int

# They enforce order
color1 = MyRGB(1, 2, 3)
color2 = MyRGB(red=1, green=2, blue=3)

# They allow type checking! (a linter would catch this)
color1.red = "two"

In [55]:
@dataclass(kw_only=True)
class MyRGB:
    red: int
    green: int
    blue: int

MyRGB(1, 2, 3)

TypeError: MyRGB.__init__() takes 1 positional argument but 4 were given

MyRGB(red=1, green=2, blue=3)

In [60]:
print(color1 == color2)
print(MyRGB(red=1, green=2, blue=3) == MyRGB(red=1, green=2, blue=3))

False
True


In [68]:
from dataclasses import asdict
asdict(color1)

{'red': 1, 'green': 2, 'blue': 3}

In [62]:
class Planet:
    def __init__(self, distance, size):
        self.distance = distance
        self.size = size

    def __repr__(self):
        return (
            f"{type(self).__module__}"
            f"{type(self).__name__}("
            f"distance={self.distance}, "
            f"size={self.size})"
        )

far = Planet(10, 5) # planet size 5, 10 away from us
near = Planet(1, 2) # planet size 2, 1 away from us
data = [far, near]
# This is going to fail because python doesn't know how to compare planets!
data.sort()

TypeError: '<' not supported between instances of 'Planet' and 'Planet'

In [63]:
class Planet:
    ...

    def _astuple(self):
        return (self.distance, self.size)

    def __eq__(self, other):
        return (
            type(self) == type(other)
            and self._astuple() == other._astuple()
        )

    def __lt__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() < other._astuple()

    def __le__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() <= other._astuple()

    def __gt__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() > other._astuple()

    def __ge__(self, other):
        if type(self) != type(other):
            return NotImplemented
        return self._astuple() >= other._astuple()

In [66]:
@dataclass(order=True)
class DataclassPlanet:
    distance: float
    size: float

far2 = DataclassPlanet(10, 2)
near2 = DataclassPlanet(1, 5)
print(far2 > near2)
print(near2 < far2)

data = [far2, near2]
data.sort()
data

True
True


[DataclassPlanet(distance=1, size=5), DataclassPlanet(distance=10, size=2)]

As you can see, `dataclass` is _pretty cool_. A possible Java equivalent might be Lombok's `Data` annotation.

Think of it as a boilerplate slayer.

In [69]:
# Pydantic is good stuff. Let's see how it compares to dataclasses.

import sys
from pydantic import BaseModel

@dataclass
class PointV1:
    x: float
    y: float

class PointV2(BaseModel):
    x: float
    y: float

point1 = PointV1(1, 2)
point2 = PointV2(x=1, y=2)
print("The dataclass is", sys.getsizeof(point1), "bytes")
print("The pydantic model is", sys.getsizeof(point2), "bytes")

The dataclass is 48 bytes
The pydantic model is 72 bytes


In [45]:
print(dir(point1))
print('-' * 80)
print(dir(point2))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__replace__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', 'x', 'y']
--------------------------------------------------------------------------------
['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__firstlineno__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__'

This kind of difference can compound at scale. Prefer dataclasses to pydantic where you _do not_ need pydantic's powerful validation tooling.

By the way: `dir` is just a cool lil function that either dumps the names in the current local scope, OR lists out all the attributes of an object. I like to use it as a fast and easy way to discover methods and fields on an object.

In [70]:

PointV2(x=1, y="two")  # dataclass would allow this at runtime

ValidationError: 1 validation error for PointV2
y
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='two', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing

# Item 52: Use `@classmethod` Polymorphism to Construct Objects Genericall
Python only has __init__

In [75]:
# First, a parent class for our mapreduce input data
class InputData:
    def read(self):
        raise NotImplementedError

# Second, a concrete impl
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

# Third, a worker
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

# Fourth, a concrete worker that counts lines in a file
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")

    def reduce(self, other):
        self.result += other.result

import os

# FIFTH, a GENERATOR for inputs
def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))

# Sixth, workers!
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

from threading import Thread

# SEvEnTh, execute!
def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

In [76]:
# Tying it all together...
def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

# kinda iffy.. this mapreduce is _pretty specific_

In [90]:
import os
import random
from tempfile import TemporaryDirectory

def write_test_files(tmpdir):
    for i in range(10000):
        with open(os.path.join(tmpdir, str(i)), "w") as f:
            f.write("\n" * random.randint(0, 100))

with TemporaryDirectory() as tmpdir:
    print("MAKING TEST FILES!")
    write_test_files(tmpdir)

    print("RUNNING MAPREDUCE!")
    result = mapreduce(tmpdir)
    print(f"There are {result} lines")


MAKING TEST FILES!
RUNNING MAPREDUCE!
There are 502154 lines


![](fry.jpg)
Okay... so whats the problem?

Well, this bad boy ain't generic at all. We made a mapreduce that only works for counting lines in a file; mapreduce is meant to be a _lot_ more generic than that.

In [91]:
# Let's generify this turd; Remember, Python only has __init__, so we can't just lean on polymorphic constructors.

# BEHOLD, STATIC CONSTRUCTORS VIA CLASSMETHOD

class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

In [92]:
# Mix up some concrete

class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config["data_dir"]
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):  # SEE <--- DO YOU SEE?? No longer does the worker need deep knowledge of the construction of inputs
            workers.append(cls(input_data))
        return workers

# Fourth, a concrete worker that counts lines in a file
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")

    def reduce(self, other):
        self.result += other.result

    # nothing else changed

In [93]:
# rewritten mapreduce
def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

In [97]:
with TemporaryDirectory() as tmpdir:
    config = {"data_dir": tmpdir}
    print("MAKING TEST FILES!")
    write_test_files(tmpdir)

    print("RUNNING MAPREDUCE!")
    result = mapreduce(LineCountWorker, PathInputData, config)
    print(f"There are {result} lines")


MAKING TEST FILES!
RUNNING MAPREDUCE!
There are 496105 lines


Things to Remember

- Python only supports a single constructor per class: the __init__ method.
- Use @classmethod to define alternative constructors for your classes.
- Use class method polymorphism to provide generic ways to build and connect many concrete subclasses.

# Item 53: Initialize Parent Classes with super

In [98]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5) # you may be tempted to do this.

In [99]:
class TimesTwo:
    def __init__(self):
        self.value *= 2

class PlusFive:
    def __init__(self):
        self.value += 5

In [100]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = OneWay(5)
print("First ordering value is (5 * 2) + 5 =", foo.value)

First ordering value is (5 * 2) + 5 = 15


In [101]:
class OneWay(MyBaseClass, PlusFive, TimesTwo): # now we'd expect the order to be (5 + 5) * 2
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = OneWay(5)
print("Second ordering should be (5 + 5) * 2, but is", foo.value)

First ordering value is (5 * 2) + 5 = 15


Why? We changed Arg order, but not call order in the `OneWay.__init__`

In [103]:
# Diamond inheritance causes problems too
class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7

class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9

class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)  # this _resets_ the value, instead of taking TimesSeven's output

foo = ThisWay(5)
print("Should be (5 * 7) + 9 = 44 but is", foo.value, "aka 5 + 9")

Should be (5 * 7) + 9 = 44 but is 14 aka 5 + 9


In [104]:
# USE SUPER
# Super makes sure common parents are initialized only once

class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 7

class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

In [108]:
class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

foo = GoodWay(5)
print("Should be 7 * (5 + 9) = 98 and is", foo.value)
mro_str = "\n".join(repr(cls) for cls in GoodWay.__mro__)
print(mro_str)

# mro lets us see the method resolution order, aka, what order inits were called.


Should be 7 * (5 + 9) = 98 and is 98
<class '__main__.GoodWay'>
<class '__main__.TimesSevenCorrect'>
<class '__main__.PlusNineCorrect'>
<class '__main__.MyBaseClass'>
<class 'object'>


In [109]:
# Once the top of the diamond is reached, the inits actually do work in the opposite order of how they were called.

# super also avoids pitfalls of renaming classes!

# But anyway this is all moot because multiple inheritance is pain, and we're going to look at a different way.

# Item 54: Consider Composing Functionality with Mix-in Classes
All the convenience and encapsulation of multiple inheritance, without the suffering!

> A mix-in is a class that defines only a small set of additional methods for its child classes to provide. Mix-in classes don’t define their own instance attributes or require their __init__ constructor to be called.

In [110]:
# The book example is really good

class ToDictMixin:
    """A mixin class that provides a to_dict method."""
    def to_dict(self):
        return self._traverse_dict(self.__dict__)

    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output

    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, "__dict__"):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [111]:
# lets use it!
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

tree = BinaryTree(
    10,
    left=BinaryTree(7, right=BinaryTree(9)),
    right=BinaryTree(13, left=BinaryTree(11)),
)
print(tree.to_dict()) # BEHOLD

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


In [116]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(
        self,
        value,
        left=None,
        right=None,
        parent=None,
    ):
        super().__init__(value, left=left, right=right)
        self.parent = parent


root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

RecursionError: maximum recursion depth exceeded

In [117]:
# Whoops!

# Well, hey, with Mix-Ins, its easy to fix! We have _very_ pluggable behavior:

class BinaryTreeWithParent(BinaryTree):
    def __init__(
        self,
        value,
        left=None,
        right=None,
        parent=None,
    ):
        super().__init__(value, left=left, right=right)
        self.parent = parent

    def _traverse(self, key, value): # override the _traverse FROM THE MIX IN to avoid the cyclic dependency
        if (
            isinstance(value, BinaryTreeWithParent)
            and key == "parent"
        ):
            return value.value  # Prevent cycles
        else:
            return super()._traverse(key, value)

In [118]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None, 'parent': 7}, 'parent': 10}, 'right': None, 'parent': None}


Brief aside for Java: instead of getting shared behavior via `abstract` in Java, use default interfaces! Java 8+ interfaces provide mixin behavior.

There is virtually no reason to use `abstract` in Java since 2013. STOP IT.  (I mean... unless you have a good reason)

In [119]:
# you can compose mixins! WOW
import json

class JsonMixin:
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)

    def to_json(self):
        return json.dumps(self.to_dict()) # DO YOU SEE????

In [120]:
class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [
            Machine(**kwargs) for kwargs in machines]

class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports=None, speed=None):
        self.ports = ports
        self.speed = speed

class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk

In [121]:
serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12},
        {"cores": 2, "ram": 4e9, "disk": 500e9}
    ]
}"""

deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()
assert json.loads(serialized) == json.loads(roundtrip)

# Mixins are the bees knees